In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# some imports from my project library
from turboflow.utils import phy_utils as phy
from turboflow.utils import dsp_utils as dsp
from turboflow.utils import torch_utils as tch
from turboflow.utils import viz_utils as viz
from turboflow import evaluation as evl


# import torch lib
import torch
# import pl torch libs
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

In [4]:
from turboflow.dataloaders import TurboFlowDataModule
args = {
    'dataset' : 'Turb2D',
    'data_dir' : None,
    'batch_size' : 100000,
    'time_idx' : 42,
    'train_downsampling' : 4,
    'val_downsampling' : 4,
    'test_downsampling' : 1
    
}
dm = TurboFlowDataModule(**args)
dm.setup()

In [5]:
for batch in dm.train_dataloader():
    X, y = batch
    print(X.shape, y.shape)

for batch in dm.val_dataloader():
    X, y = batch
    print(X.shape, y.shape)
    print(X[:2,:])
    print(X[2,1]-X[2,0])
    
for batch in dm.test_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)
    print(X[:2,:])
print('%1.8f' % (X[2,1]-X[2,0]))

torch.Size([4096, 2]) torch.Size([4096, 2])
torch.Size([4096, 2]) torch.Size([4096, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0157]])
tensor(0.0314)
torch.Size([65536, 2])
torch.Size([65536, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0039]])
0.00784314


In [6]:
from turboflow.models.phyrff_hard import plDivFreeRFFNet

args = {
    'name':'DivFreeNetRFF',
    'mlp_layers_dim' : 256,
    'mlp_layers_num' : 3,
    'mlp_last_actfn': 'tanh',
    'do_rff' :   True, 
    'rff_num' : 256,
    'rff_scale' :  10,
    'lam_pde': 1e-4,
    'lam_reg': 0,
    'lam_sfn': 0,
    'sfn_min_x':0.00784314,
    'sfn_num_centers'   :50,
    'sfn_num_increments':3,
    'sfn_patch_dim':32,
}

model = plDivFreeRFFNet(**args)

In [7]:
x = torch.randn(10,2)
model(x)

(tensor([[ 2.0099, -3.4388],
         [-5.3524, -1.2877],
         [ 0.4447,  4.3464],
         [-2.5105,  3.3002],
         [-3.1376,  1.7063],
         [-7.5726, -8.3330],
         [ 0.8607, -1.1402],
         [-2.3013, -1.3965],
         [ 4.9681,  3.6961],
         [ 4.2605,  1.8359]], grad_fn=<CatBackward>),
 tensor([[-0.1114],
         [ 0.0131],
         [-0.0046],
         [-0.1198],
         [-0.1093],
         [-0.1660],
         [-0.1318],
         [-0.0112],
         [-0.2841],
         [-0.1640]], grad_fn=<TanhBackward>))

In [ ]:
early_stop_callback = EarlyStopping(monitor='val_loss')

trainer = pl.Trainer(gpus=1, 
                     check_val_every_n_epoch=200, 
                     max_epochs=5000, 
                     callbacks=[early_stop_callback])

trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/dicarlo_d/Documents/Code/TurboSuperResultion/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-10-15 13:55:58.504979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-15 13:55:58.505013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

  | Name | Type    | Params
---------------------------------
0 | rff  | Fourier | 512   
1 | mlp  | MLP     | 263 K 
2 | div  | DivFree | 

Validation sanity check: 0it [00:00, ?it/s]

/home/dicarlo_d/Documents/Code/TurboSuperResultion/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]